In [30]:
import json
import pandas as pd 
from datetime import datetime
import os
def parse_date(date):
    if "," in date:
        return datetime.strptime(date, "%b %d, %Y")
    elif len(date)==4:
        return datetime.strptime(date, "%Y")
    else:
        return datetime.strptime(date, "%b %Y")
folder = "../input/crunchbase/retail-over-time"
files = os.listdir(folder)


In [138]:
processed_data = []
for file in files:
    with open(folder+"/"+file, 'r') as file:
        print(file)
        data = json.load(file)
    
    header = data[0]
    count = 0
    for i, line in enumerate(data):
        if line[0] == "Organization Name":
            continue
        processed_data.append(line)

    


df = pd.DataFrame(processed_data)
df.columns = [x.lower() for x in header]
df["founded date"] = df["founded date"].apply(lambda x: parse_date(x))



<_io.TextIOWrapper name='../input/crunchbase/retail-over-time/log (3).json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../input/crunchbase/retail-over-time/log (1).json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../input/crunchbase/retail-over-time/log (2).json' mode='r' encoding='UTF-8'>


In [139]:
import numpy as np
def fix(x):
    parts = x.split(",")
    split_index = next((i for i, part in enumerate(parts) if len(part) > 40 or part == "—"), None)
    industries = ", ".join(parts[:split_index])
    description = ", ".join(parts[split_index:-1])
    return [industries, description]
   
df["headquarters location"] = list(df.industries.apply(lambda x: x.split(",")[-1]) + ", " + df["full description"] + ", " + df["headquarters location"])
df[["industries","full description"]] = list(df.industries.apply(lambda x: fix(x)))

In [140]:
df['description_all'] = df['description'] + " " + df['full description']

In [146]:
df = df.sort_values(by="founded date")
df = df.drop_duplicates(subset=['organization name'], keep='first')
df = df.reset_index(drop=True)
df.to_csv("../output/data/retail-data.csv")


In [147]:
len(df)

13159

In [5]:
pre = df[pd.to_datetime(df['founded date'])<'11/30/2022']
post = df[pd.to_datetime(df['founded date'])>='11/30/2022']
len(pre)

1000